In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*Final Assignment Sample Code*

In [43]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

iris_dict = load_iris()
X = iris_dict['data']
y = iris_dict['target']

# shuffle arrays since y values are in order

from sklearn.utils import shuffle
X_new, y_new = shuffle(X, y, random_state=0)

# Divide samples into train and test 

n_samples_train = 120 # number of samples for training (--> #samples for testing = len(y_new) - 120 = 30)
X_train = X_new[:n_samples_train, :]
y_train = y_new[:n_samples_train]

X_test = X_new[n_samples_train:, :]
y_test = y_new[n_samples_train:]

clf = LogisticRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

import pickle

with open('iris_trained_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

with open('iris_trained_model.pkl', 'rb') as f:
    clf_loaded = pickle.load(f)

clf_loaded

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
!pip install mysqlclient

     |████████████████████████████████| 92kB 3.8MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.0.3-cp37-cp37m-linux_x86_64.whl size=100113 sha256=d57585ad8b59133c14d4b789a7ff6d8644bbedaaed51bf74f3e6316dee1b4bde
  Stored in directory: /root/.cache/pip/wheels/75/ca/e8/ad4e7ce3df18bcd91c7d84dd28c7c08db491a2a2360efed363
Successfully built mysqlclient


*Defining Airflow Directory and Creating airflow.cfg*

In [15]:
!export AIRFLOW_HOME=~/airflow

*Initialization*

In [48]:
!airflow db init

DB: sqlite:////root/airflow/airflow.db
[2021-06-13 14:47:26,018] {db.py:695} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [airflow.models.crypto] empty cryptography key - values will not be stored encrypted.
Initialization done


 *My Main Code*

In [67]:
# Code by Hamide Hadibeik
# .................... Import Packages ...........................

from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.utils import timezone
from datetime import datetime, timedelta

import glob
import pandas as pd
import csv

import pickle
import boto3

from sklearn.linear_model import LogisticRegression

# ..................... AmazonS3Client Class .......................

class AmazonS3Client(object):
    _cl = None
    def __init__(self):
        if not self._cl:
            self._cl = boto3.client('S3',
                aws_access_key_id='ASIAWZXXBZXUBCOXJD6',
                aws_secret_access_key='vqGoB5IgLjmQP4r+H7Y+CivkZ9MDFnDrL9wGeJ4',
                aws_session_token='FwoGZXIvYXdzEGAaDE+MCg9UQ4hn0kVFUCLIARC3cCH2IfPKhdrjgcUDCohmMQNlkRfjNWs0lltcqFZ1UnNlFISqfIbufsn6H/4a9gUEXcR0yM04LM0F65e9Wo+CpGJtwlg96JK/kCftDmG+lt9zvJukXPKdwb7wAzWt/7f+aNz3O9ohpctEkK16FmltPD3+bICxgPxKqiLdNYKnBZ4XEbPGA0/1PySJIqsPgjP4nRtfMGKJTvDHsfsLjiwFzuhmeypoxGPjBDNFVrDMUOvEUqpE/JQXJu6D7USlklTm44Yn14noKNH87PYFMi1kxz8FAIx5Tl1TukfahpGSt+tco7UCZuIG26DopnkXMSP/VT8cBXzgpsM/FF=',
            )
        self.cl = self._cl

# ...................... Download and Upload ....................

def download(file_name, bucket_name, save_as=None):
    s3 = AmazonS3Client().cl
    if not save_as:
        save_as = file_name
    s3.download_file(bucket_name, file_name, save_as)


def upload(file_obj, file_name, bucket_name):
    s3 = S3ClientSingleton().cl
    s3.put_object(
        Bucket=bucket_name,
        Key=file_name,
        Body=file_obj)


#.............. Task functions (task1, task2, task3, task4) .............

def task1():
    s3 = S3ClientSingleton().cl
    for s3_samp in s3.list_objects(Bucket='hamideh.ub.training')['Contents']:
        filename = s3_samp['Key']
        download(filename, 'hamideh.ub.training', filename)


def task2():
    file = glob.glob("train*.csv")
    data = []
    for myfile in file:
        df = pd.read_csv(myfile, index_col=None, header=0)
        data.append(df)
    df = pd.concat(data, axis=0, ignore_index=True)
    X = df.loc[:, df.columns != 'Species']
    y = df.loc[:, df.columns == 'Species']
    clf = LogisticRegression()
    clf.fit(X, y)
    with open('iris_trained_model.pkl', 'wb') as f:
        pickle.dump(clf, f)

def task3():
    download('predict.csv', 'hamideh.ub.prediction')
    my_test_data = pd.read_csv('predict.csv', index_col=None, header=0)
    with open('iris_trained_model.pkl', 'rb') as f:
        clf_loaded = pickle.load(f)
    y_pred = clf_loaded.predict(my_test_data)
    my_test_data['Species'] = y_pred
    my_test_data.to_csv('_' + 'predict.csv')

def task4():
    with open('iris_trained_model.pkl', 'rb') as f1:
        upload(f1, 'iris_trained_model.pkl', 'hamideh.ub.ml')
    with open('predict.csv', 'rb') as f2:
        myfiles = 'prediction_{}'.format(datetime.today())
        upload(f2, myfiles, 'hamideh.ub.prediction')


#............... Arguments of the DAG ...................

args = {
    'owner': 'Hamideh',
    'depends_on_past': False,
    'start_date': timezone.utcnow(),
    'email': ['airflow@my_first_dag.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5)
}

# ................ My DAG ...................

dag = DAG(dag_id='MyDag',
          default_args=args,
          description="MyDAG in Final Assignment of This Course",
          schedule_interval='0 20 0 * 1')  # at 8 PM each Monday 

# ................... Tasks 1-4 .....................

op1 = PythonOperator(task_id='Task1',
    python_callable=task1,
    dag=dag)

op2 = PythonOperator(task_id='Task2',
    python_callable=task2,
    dag=dag)

op3 = PythonOperator(task_id='Task3',
    python_callable=task3,
    dag=dag)

op4 = PythonOperator(task_id='Task4',
    python_callable=task4,
    dag=dag)


# ........................ End ................................


*Create DAG*

In [68]:
op1 >> op2

<Task(PythonOperator): Task2>

In [69]:
[op2, op3] >> op4

<Task(PythonOperator): Task4>

*Connecting to localhost:8081*

In [55]:
!airflow webserver -p 8081

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
[2021-06-13 15:02:16,938] {dagbag.py:487} INFO - Filling up the DagBag from /dev/null
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:8081
Timeout: 120
Logfiles: - -
Access Logformat: 
[2021-06-13 15:02:19 +0000] [4126] [INFO] Starting gunicorn 20.1.0
[2021-06-13 15:02:19 +0000] [4126] [INFO] Listening at: http://0.0.0.0:8081 (4126)
[2021-06-13 15:02:19 +0000] [4126] [INFO] Using worker: sync
[2021-06-13 15:02:19 +0000] [4129] [INFO] Booting worker with pid: 4129
[2021-06-13 15:02:19 +0000] [4130] [INFO] Booting worker with pid: 4130
[2021-06-13 15:02:19 +0000] [4131] [INFO] Booting worker with pid: 4131
[2021-06-13 15:02:19 +0000] [4132] [INFO] Booting worker with pid: 4132
[2021-06-13 15:02:30,879] {webserver_command.py:431} INFO - Rece

In [49]:
!airflow scheduler

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
Starting flask
 * Serving Flask app "airflow.utils.serve_logs" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
[2021-06-13 14:48:16,879] {_internal.py:122} INFO -  * Running on http://0.0.0.0:8793/ (Press CTRL+C to quit)
[2021-06-13 14:48:16,887] {scheduler_job.py:1253} INFO - Starting the scheduler
[2021-06-13 14:48:16,888] {scheduler_job.py:1258} INFO - Processing each file at most -1 times
[2021-06-13 14:48:16,893] {dag_processing.py:254} INFO - Launched DagFileProcessorManager with pid: 3946
[2021-06-13 14:48:16,895] {scheduler_job.py:1822} INFO - Resetting orphaned tasks for active dag runs
[2021-06-13 14:48:16,901] {settings.py:52} INFO - Configured default timezone Timezone('UTC')
[2021-06-13 1

*Checking the Time According to UTC*

In [50]:
from airflow.utils import timezone

now = timezone.utcnow()
print(now)


2021-06-13 14:48:41.508334+00:00
